In [2]:
# Importare le librerie
import tweepy
from dotenv import dotenv_values
# Importare le variabili d'ambiente dal file .env
config = dotenv_values(".env")

Definiamo le funzioni serealize_json e read_json che ci serviranno rispettivamente per salvare dati in locale e leggere file JSON da locale, servendoci delle librerie json e os.

In [3]:
# %load utils.py
import json
import os


def serealize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}.json")


def read_json(path):
    try:
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as file:
                data = json.load(file)
            return data
    except ValueError:
        print("Path not found, check the correctness of the path")


Utilizzo la libreria Tweepy per creare un client. Esso interagirà con la Twitter API. La libreria Tweepy permette di astrarre le chiamate HTTP per interagire con la Twitter API semplificandone l'utilizzo.


Creiamo un client Tweepy utilizzando il `BREARER_TOKEN` presente nel file `.env` e caricato tramite la libreria `dotenv`.


In [4]:
# Nuovo client Tweepy
client = tweepy.Client(
    bearer_token=config["BEARER_TOKEN"], wait_on_rate_limit=True)


Creo una funzione generica get_parsed_users_followers(id) che mi restituisce un dizionario di tutti i follower di un utente e per ciascuno di essi le seguenti informazioni salvate: attributi di default, descrizione del profilo, metriche pubbliche e se l'account è protetto. 
Questa funzione verrà spesso utilizzata successivamente.

In [5]:
def get_parsed_users_followers(id):
    followers = {}
    paginator = tweepy.Paginator(client.get_users_followers, id=id, user_fields=[
                                 "public_metrics", "protected", "description"], max_results=1000, limit=5).flatten(limit=1000)
    for follower in paginator:
        found_follower = {
            'name': follower.name,
            'username': follower.username,
            'description': follower.description,
            'public_metrics': follower.public_metrics,
            "protected": follower.protected
        }
        followers[follower.id] = found_follower
    return followers


## Punto 1
Usando la libreria tweepy e la funzione get_parsed_users_followers(id) da noi definita nel punto precedente scarico tutti i follower del profilo twitter dell'utente con username @KevinRoitero, ciascuno con le seguenti informazioni: attributi di default, la descrizione del profilo, le metriche pubbliche e se l'account è protetto o no.


In [ ]:
kevin = client.get_user(username="KevinRoitero", user_fields=["public_metrics", "protected", "description"]).data
followers = get_parsed_users_followers(kevin.id)

# print(f'Followers count: {len(followers)}')
serealize_json("out", "followers", followers)


## Punto 2
Per ciascun follower del profilo di @KevinRoitero utilizziamo tweepy per scaricare il numero di tweet pubblicati durante gli ultimi 7 giorni e aggiungiamo il risultato al JSON prodotto al punto 1.

In [ ]:
for id in followers:
    last_week_tweets_count = 0
    recent_tweets_count = client.get_recent_tweets_count(
        query=f"from:{followers[id]['username']} -is:retweet", granularity="day").data
    for day in recent_tweets_count or []:
        last_week_tweets_count += day["tweet_count"]
    # aggiungo alle info di follower anche last_week_tweets_count
    followers[id]['last_week_tweets_count'] = last_week_tweets_count
# Aggiungere il numero di tweets nel JSON
serealize_json("out", "followers", followers)


## Punto 3


In [19]:
# filtro la lista di follower di kevin scegliendo solo i follower che hanno a loro volta almeno un follower e non sono protected
follower_id_filtrati = []

for id in followers:
    follower_count = followers[id]["public_metrics"]["followers_count"]
    protected = followers[id]["protected"]

    if (follower_count > 1 and protected == False):
        follower_id_filtrati.append(id)


In [ ]:
for id in follower_id_filtrati:
    sub_followers = get_parsed_users_followers(id)
    followers[id]["followers"] = sub_followers

serealize_json("out", "followers", followers)

## Punto Pre-4
Dato che il punto 3 impega circa 2 ore per generare tutti i dati, li abbiamo salvati all'interno di un file JSON. Per leggere tale file usiamo la funzione `read_json()` 

In [1]:
followers = read_json("./out/followers_completed.json")

NameError: name 'read_json' is not defined

## Punto 4


In [8]:
import networkx as nx

In [9]:
def get_follower_attributes(id):
    return {
        'username': followers[id]['username'],
        'description': followers[id]['description'],
        'followers_count': followers[id]['public_metrics']['followers_count']
    }

In [10]:
graph1 = nx.DiGraph()

graph1.add_node(kevin.id,attr={
    'username': kevin.username,
    'description': kevin.description,
    'followers_count': kevin.public_metrics['followers_count']
})
for id in followers:
    graph1.add_node(id, attr=get_follower_attributes(id))


In [11]:
# Check if A follow B
def follow_check(A, B):
    if ("followers" in followers[B]):
        if A in followers[B]["followers"]:
            return True
        return False


for A in followers:
    graph1.add_edge(A,kevin.id)

for A in followers:
    for B in followers:
        if follow_check(A, B):
            graph1.add_edge(A,B)


## Punto 5

In [12]:
graph2 = graph1.to_undirected()
graph2 = nx.barabasi_albert_graph(n = graph2.number_of_nodes()*2, m = 2, initial_graph=graph2)

## Punto 6

Grafici Grafo 1

In [ ]:
# Grafico interattivo
import pyvis

visual_graph1 = pyvis.network.Network(height="100vh", notebook=True)
visual_graph1.barnes_hut()
visual_graph1.from_nx(graph1)
visual_graph1.show("graphs/grafico1_interattivo.html")


# Grafico statico
from netgraph import Graph
import matplotlib.pyplot as plt

d = dict(graph1.degree)
for x in d:
    d[x] = float(d[x])
    d[x] = d[x]/30

Graph(
    graph = graph1,
    arrows=True,
    edge_width=0.5,
    node_size=d,
    node_layout="spring",
)

fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig("graphs/grafico1_statico.pdf", dpi=100)
plt.show()

Grafici Grafo 2

In [ ]:
# Grafico interattivo
visual_graph2 = pyvis.network.Network(height="100vh", notebook=True)
visual_graph2.barnes_hut()
visual_graph2.from_nx(graph2)
visual_graph2.show("graphs/grafico2_interattivo.html")


# Grafico statico
from netgraph import Graph
import matplotlib.pyplot as plt

d = dict(graph2.degree)
for x in d:   
    d[x] = float(d[x])
    d[x] = d[x]/30
    
Graph(
    graph = graph2,
    arrows=False,
    edge_width=0.5,
    node_size=d,
    node_layout="spring"
)

fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig("graphs/grafico2_statico.pdf", dpi=100)
plt.show()

## Punto 7

In [ ]:
def grafico(graph,scc):
    d = dict(graph.degree)
    for x in d:
        d[x] = float(d[x])
        d[x] = d[x]/30

    nodes = graph.nodes
    node_colors = dict()
    for node in nodes:
        if node in scc:
            node_colors[node] = 'red'
        else:
            node_colors[node] = 'black'
    Graph(
        graph=graph,
        edge_width=0.5,
        node_size=d,
        node_color=node_colors,
        node_layout="spring",
    )

scc1 = max(nx.strongly_connected_components(G=graph1), key=len)
# scc2 = max(nx.strongly_connected_components(G=graph2), key=len)
scc2 = max(nx.connected_components(G=graph2), key=len)

# TODO: Salvare i grafici in un file
plt.figure(1)
grafico(graph1,scc1)
plt.figure(2)
grafico(graph2,scc2)
plt.show()


## Punto 8

In [ ]:
# TODO: Solito problema, non funziona nulla
print(list(nx.radius(graph1)))

## Punto 9

In [21]:
def draw(graph, pos, measures, measure_name):
    nodes = nx.draw_networkx_nodes(
        graph,
        pos,
        node_size=150,
        cmap=plt.cm.plasma,
        node_color=list(measures.values()),
        nodelist=measures.keys()
    )
    edges = nx.draw_networkx_edges(graph,pos)
    plt.title(measure_name)
    plt.colorbar(nodes)
    plt.axis('off')

In [ ]:
# TODO: Questi dati devono essere mostrati tutti in diversi grafici
def centrality(graph):
    plt.figure()
    draw(
        graph,
        nx.spring_layout(graph),
        nx.betweenness_centrality(graph),
        'Betweenness Centrality'
    )
    plt.figure()
    draw(
        graph,
        nx.spring_layout(graph),
        nx.closeness_centrality(graph),
        'Closeness Centrality'
    )
    plt.figure()
    draw(
        graph,
        nx.spring_layout(graph),
        nx.degree_centrality(graph),
        'Degree Centrality'
    )
    if nx.is_directed(graph):
        plt.figure()
        draw(
            graph,
            nx.spring_layout(graph),
            nx.in_degree_centrality(graph),
            'In-Degree Centrality'
        )
    if nx.is_directed(graph):
        plt.figure()
        draw(
            graph,
            nx.spring_layout(graph),
            nx.out_degree_centrality(graph),
            'Out-Degree Centrality'
        )
    plt.figure()
    draw(
        graph,
        nx.spring_layout(graph),
        nx.pagerank(graph),
        'Page Rank'
    )
    plt.figure()
    # TODO: Grafico HITS non funziona
    draw(
        graph,
        nx.spring_layout(graph),
        nx.hits(graph),
        'HITS'
    )
    plt.show()

# TODO: Salvare i grafici in un file
centrality(graph1)
centrality(graph2)

## Punto 10

In [35]:
omega = nx.omega(graph2)
sigma = nx.sigma(graph2)
print('small-world coefficient')
print(f'Omega: {omega}\nSigma: {sigma}')